# Inports

Usual imports.


In [1]:
import numpy as np
import math
import pygame
import os

# Constants

Important constants for the game GO to improve code readability later on.

In [2]:
black = -1
white = 1
empty = 0

display_w = 900
display_h = 900

WHITE = (255,255,255)
BLACK = (0,0,0)
RED = (210, 83, 83)
GREY = (200, 200, 200)
DARK_GREY = (75, 75, 75)
BROWN = (102, 51, 0)

line_width = 3
hoshi_r = 7

komi = 5.5

# Inits


In [3]:
pygame.init()
screen = pygame.display.set_mode((display_w, display_h))
pygame.display.set_caption('GO')
clock = pygame.time.Clock()

In [4]:
text = pygame.font.SysFont('monospace', 44)
background = pygame.image.load(os.path.join("textures","wood.jpg")).convert_alpha()

bs9 = pygame.image.load(os.path.join("textures","black9.png")).convert_alpha()
bs13 = pygame.image.load(os.path.join("textures","black13.png")).convert_alpha()
bs19 = pygame.image.load(os.path.join("textures","black19.png")).convert_alpha()

ws9 = pygame.image.load(os.path.join("textures","white9.png")).convert_alpha()
ws13 = pygame.image.load(os.path.join("textures","white13.png")).convert_alpha()
ws19 = pygame.image.load(os.path.join("textures","white19.png")).convert_alpha()

# Classes

Definitions of useful classes

In [5]:
class Board:
    game_board = None
    turn = 0
    passed = False
    finished = False
    size = None
    last_move = None
    
    def __init__(self, size):
        self.game_board = np.zeros((size,size), dtype=np.int8)
        self.size = size
        
    def get_situation(self):
        return self.game_board
    
    def get_board_size(self):
        return self.size
    
    def get_turn(self):
        return self.turn
    
    def get_player(self):
        if self.turn % 2 == 0:
            return black
        else:
            return white
        
    def make_move(self, row=None, col=None, _pass=False):
        if self.finished:
            return
        elif _pass:
            self.turn += 1
            
            if self.passed:
                self.finished = True
            
            self.passed = True
            return
        elif self.game_board[row,col] != empty:
            return "Invalid move!"
        else:
            self.passed = False
            
            self.game_board[row,col] = self.get_player()
            self.turn += 1
            self.last_move = (row,col)
            return

# Game Functions

Functions to draw and play the game

### 1. Functions to convert pixel coordinates to array indices and the other way around.

In [6]:
def intersection_to_pixel(size, i, j):
    if(size == 9):
        void = 88
    elif(size == 13):
        void = 61
    elif(size == 19):
        void = 42
        
    x = 50 + int(void/2) + j*void
    y = 50 + int(void/2) + i*void
    
    return (x,y)

In [7]:
def pixel_to_intersection(size, x, y):
    if(size == 9):
        void = 88
    elif(size == 13):
        void = 61
    elif(size == 19):
        void = 42
    
    i = None
    j = None
    
    for k in range(size):
        if(x <= 50 + void + k*void and j is None):
            j = k
        if(y <= 50 + void + k*void and i is None):
            i = k
            
    return i, j    

### 2. Functions to draw stuff on the screen.

In [8]:
def draw_hoshis(size, screen):
    if(size == 9):
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 2, 2), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 6, 6), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 2, 6), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 6, 2), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 4, 4), hoshi_r, 0)
    elif(size == 13):
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 3, 3), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 9, 9), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 3, 9), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 9, 3), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 6, 6), hoshi_r, 0)
    elif(size == 19):
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 3, 3), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 15, 15), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 3, 15), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 15, 3), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 9, 9), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 9, 15), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 15, 9), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 9, 3), hoshi_r, 0)
        pygame.draw.circle(screen, GREY, intersection_to_pixel(size, 3, 9), hoshi_r, 0)

In [9]:
def print_(screen, txt):
    txt_img = text.render(str(txt), 1, WHITE)
    pygame.draw.rect(screen, DARK_GREY, (0,850,900,50), 0)
    screen.blit(txt_img, (display_w/2- int(txt_img.get_rect().width/2), 850))

In [10]:
def draw_turn(board, screen):
    turn = text.render(str(board.turn), 1, WHITE)
    pygame.draw.rect(screen, DARK_GREY, (0,0,900,50), 0)
    screen.blit(turn, (display_w/2- int(turn.get_rect().width/2), 0))

In [11]:
def draw_board(board, screen):
    pygame.draw.rect(screen, DARK_GREY, (0,0,900,900), 0)
    screen.blit(background, (50,50))
    
    for i in range(board.size):
        pygame.draw.line(screen, GREY, intersection_to_pixel(board.size, i, 0), intersection_to_pixel(board.size, i, board.size-1), line_width)
        pygame.draw.line(screen, GREY, intersection_to_pixel(board.size, 0, i), intersection_to_pixel(board.size, board.size-1, i), line_width)
    
    draw_hoshis(board.size, screen)
    
    if(board.size==9):
        bs = bs9
        ws = ws9
    elif(board.size==13):
        bs = bs13
        ws = ws13
    elif(board.size==19):
        bs = bs19
        ws = ws19
    
    stone_rad = int(bs.get_rect().width/2)
    sit = board.get_situation()
    
    for i in range(board.size):
        for j in range(board.size):
            if(sit[i,j] == black):
                screen.blit(bs, np.subtract(intersection_to_pixel(board.size, i,j), (stone_rad, stone_rad)))
            elif(sit[i,j] == white):
                screen.blit(ws, np.subtract(intersection_to_pixel(board.size, i,j), (stone_rad, stone_rad)))
                
    last_move = board.last_move
    
    if(not (last_move is None or board.passed)):
        xy = intersection_to_pixel(board.size, last_move[0], last_move[1])

        if(board.get_player() == black):
            pygame.draw.circle(screen, white, (xy[0], xy[1]), 9, 1)
        elif(board.get_player() == white):
            pygame.draw.circle(screen, black, (xy[0], xy[1]), 9, 1)        

## Gameloop
The following function is the main game loop.
This is where the magic happens.

* the game situation is drawn
* inputs of the players are recorded


In [12]:
def gameloop(board):   
    beendet = False
    draw_board(board, screen)
    err = None
    
    while not board.finished and not beendet:
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                beendet = True
                
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    beendet = True    
                if event.key == pygame.K_SPACE:
                    board.make_move(_pass=True)
                    
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_buttons = pygame.mouse.get_pressed()
                if mouse_buttons[0]:
                    pos = pygame.mouse.get_pos()
                    if(pos[0] >= 50 and pos[1] >= 50 and pos[0] <= display_w-50 and pos[1] <= display_h-50):
                        i, j = pixel_to_intersection(board.size, pos[0], pos[1])
                        err = board.make_move(row=i, col=j)
    
                        
        
        draw_board(board, screen)
        draw_turn(board, screen)
        
        if(type(err) is str):
            print_(screen, err)
        
        err is None
        pygame.display.update()
        clock.tick(60)
        
    
    
    
    
    
    ###
    # Block to handle quitting and finishing a game
    ###
    
    if beendet:   
        pygame.quit()
    elif board.finished:
        while not beendet:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    beendet = True
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        beendet = True
        pygame.quit()

#  Main

In here the size of the board will be determined and the main game loop will be started accordingly.

In [13]:
def main():
    board = Board(19)
    
    gameloop(board)

if __name__ == "__main__":
    main()